In [96]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as sp
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn import cross_validation
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing

import sklearn as skl
import pickle
import csv
import re
import json
import string

In [97]:
# Load Gensim features
gensim_features = np.load("gensim_features.npy")
imgs = np.load("imgs.npy")
captions = np.load("captions.npy")
unique_lab = np.load("unique_lab.npy")
output = np.load("output.npy")

In [121]:
# Threshold gensim features
threshold = .6
labels_10k = []
labels_scores = []
for image in gensim_features:
    img_idx = np.argmax(image)
    labels_10k.append(unique_lab[np.argmax(image)])
    labels_scores.append(np.max(image))
    
mask = np.where(np.array(labels_scores) > threshold)[0]
print 'Using ', len(mask), ' from 10k with threshold ', threshold

labels_10k = np.array(labels_10k)[mask]
for r in xrange(len(labels_10k)):
    if labels_10k[r] == 'video store':
        labels_10k[r] = 'videostore'
        
# When training full model, use this version
# full_indices = np.random.randint(0,output.shape[0], size=1500)

# When cross validating, use this version
# missing_labels =  set(output) - set(labels_10k)
# full_indices = []
# for m in missing_labels:
#     full_indices.append(np.where(output == m)[0][0])
    
labels_10k_full = np.concatenate([labels_10k, output])
# assert len(pd.unique(labels_10k_full)) == 200

Using  1046  from 10k with threshold  0.6


In [122]:
# alexnet features
alex_net_feat_10k = np.load("CS5785-final-data/alexnet_feat_10k.npy")
alex_net_feat_10k = alex_net_feat_10k[mask]
alexnet_train_imgs = np.load("CS5785-final-data/alexnet_feat_train.npy")
alexnet_feat_test = np.load("CS5785-final-data/alexnet_feat_test.npy")

alexnet_feat_train = np.load("CS5785-final-data/alexnet_feat_train.npy")
#with 10k data
alexnet_feat_train = np.vstack((alex_net_feat_10k, alexnet_feat_train))
#only 3k data
alexnet_feat_train_orig = np.load("CS5785-final-data/alexnet_feat_train.npy")

# sift features
sift_feat_10k = np.load("CS5785-final-data/SIFTBoW_10k.npy")
sift_feat_10k = sift_feat_10k[mask]
sift_feat_train = np.load("CS5785-final-data/SIFTBoW_train.npy")
sift_feat_test = np.load("CS5785-final-data/SIFTBoW_test.npy")
sift_feat_train = np.vstack((sift_feat_10k, sift_feat_train))

# attributes features
def read_att(file):
    with open(file, 'r') as f:
        reader = csv.reader(f, dialect='excel', delimiter='\n')
        img = []
        fv = []
        for row in reader: 
            img.append(row[0].split()[0])
            fv.append(map(int, row[0].split()[1].split(",")))
    return img, fv

_, attributes_feat_train = read_att("CS5785-final-data/attributes_train.txt")
_, attributes_feat_test = read_att("CS5785-final-data/attributes_test.txt")
attributes_feat_train = np.array(attributes_feat_train)
attributes_feat_test = np.array(attributes_feat_test)


In [150]:
# gensim attributes features
gensim_attributes_test = np.load("gensim_attributes_test.npy")
gensim_attributes_train = np.load("gensim_attributes_train.npy")

In [152]:
print gensim_attributes_test.shape
print gensim_attributes_train.shape


(1000, 200)
(3000, 200)


In [126]:
alexnet_feat_train.dtype

dtype('float64')

### Models

In [125]:
alexnet10k_full_feat_test.shape

(702, 4096)

In [124]:
#with word to vec .33
#without word2vec .32

##threshold .6
alexnet_3k_log = SGDClassifier(loss='log', penalty='l1', n_jobs=-1, n_iter=10, random_state=1)

an3k_feat_train, an3k_feat_test, an3k_y_train, an3k_y_test = train_test_split(alexnet_feat_train, labels_10k_full, test_size=0.33, random_state=42)
alexnet_3k_log.fit(an3k_feat_train, an3k_y_train)
pred_an3k = alexnet_3k_log.predict(an3k_feat_test)
print np.sum(pred_an3k == an3k_y_test) / float(len(an3k_y_test))

0.333083832335


In [114]:
from sklearn.ensemble import RandomForestClassifier
att_3k_fullclass = RandomForestClassifier(n_estimators=300, max_depth=100, random_state=1)
att_feat_train, att_feat_test, att_y_train, att_y_test = train_test_split(attributes_feat_train, output, test_size=0.33, random_state=42)
att_3k_fullclass.fit(att_feat_train, att_y_train)
pred_att_3k_fullclass = att_3k_fullclass.predict(att_feat_test)
np.sum(pred_att_3k_fullclass == att_y_test) / float(len(att_y_test))

0.2686868686868687

## models

In [161]:
#alexnet 3k rfc model
from sklearn.ensemble import RandomForestClassifier as RFC

alexnet_3k_rfc = RFC(n_estimators = 400 )
# attributes rfc
att_3k_rfc = RFC(n_estimators = 400)

In [6]:
# alexnet 3k model
alexnet_3k_log = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1)


# sift 3k model
sift_3k_log = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1)

# alexnet 10k + full class
alexnet_10k_fullclass = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1)

# sift 10k + full class
sift_10k_fullclass = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1)

# attributes
att_3k_fullclass = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1)

# gensim attributes
gensim_att_3k_fullclass = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1)

### Att 3k fullclass

In [7]:
# train models
att_feat_train, att_feat_test, att_y_train, att_y_test = train_test_split(attributes_feat_train, output, test_size=0.33, random_state=42)
att_3k_fullclass.fit(att_feat_train, att_y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=-1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [8]:
pred_att_3k_fullclass = att_3k_fullclass.predict(att_feat_test)

In [9]:
np.sum(pred_att_3k_fullclass == att_y_test) / float(len(att_y_test))

0.21414141414141413

In [10]:
att_3k_fullclass.fit(attributes_feat_train, output)

pickle.dump( att_3k_fullclass, open( "att_3k_fullclass.p", "wb" ) )

### Gensim att 3k fullclass

In [13]:
# train models
gatt_feat_train, gatt_feat_test, gatt_y_train, gatt_y_test = train_test_split(gensim_attributes_train, output, test_size=0.33, random_state=42)
gensim_att_3k_fullclass.fit(gatt_feat_train, gatt_y_train)
pred_gatt_3k_fullclass = gensim_att_3k_fullclass.predict(gatt_feat_test)
print np.sum(pred_gatt_3k_fullclass == gatt_y_test) / float(len(gatt_y_test))

0.0424242424242


In [12]:
gensim_att_3k_fullclass.fit(attributes_feat_train, output)
pickle.dump( att_3k_fullclass, open( "gensim_att_3k_fullclass.p", "wb" ) )

### alexnet_3k_log

In [14]:
# train models
an3k_feat_train, an3k_feat_test, an3k_y_train, an3k_y_test = train_test_split(alexnet_feat_train, output, test_size=0.33, random_state=42)
alexnet_3k_log.fit(an3k_feat_train, an3k_y_train)
pred_an3k = alexnet_3k_log.predict(an3k_feat_test)
print np.sum(pred_an3k == an3k_y_test) / float(len(an3k_y_test))

0.252525252525


In [28]:
alexnet_3k_log.fit(alexnet_feat_train, output)
pickle.dump( alexnet_3k_log, open( "alexnet_3k_fullclass.p", "wb" ) )

### sift_3k_log

In [16]:
sift3k_feat_train, sift3k_feat_test, sift3k_y_train, sift3k_y_test = train_test_split(sift_feat_train, output, test_size=0.33, random_state=42)
sift_3k_log.fit(sift3k_feat_train, sift3k_y_train)
pred_sift3k = sift_3k_log.predict(sift3k_feat_test)
print np.sum(pred_sift3k == sift3k_y_test) / float(len(sift3k_y_test))

0.069696969697


In [17]:
sift_3k_log.fit(sift_feat_train, output)
pickle.dump(sift_3k_log, open( "sift_3k_fullclass.p", "wb" ) )

### alexnet 10k + full class

In [20]:
alexnet10k_full_feat_train, alexnet10k_full_feat_test, alexnet10k_full_y_train, alexnet10k_full_y_test = \
        train_test_split(alex_net_feat_10k, labels_10k_full, test_size=0.33, random_state=42)
alexnet_10k_fullclass.fit(alexnet10k_full_feat_train, alexnet10k_full_y_train)
pred_alexnet_10k_full = alexnet_10k_fullclass.predict(alexnet10k_full_feat_test)
print 'Accuracy on 10k train: ', np.sum(pred_alexnet_10k_full == alexnet10k_full_y_test) / \
        float(len(alexnet10k_full_y_test))

pred_alexnet_10k_full = alexnet_10k_fullclass.predict(alexnet_feat_train)
print 'Accuracy on 3k train: ', np.sum(pred_alexnet_10k_full == output) / float(len(output))

Accuracy on 10k train:  0.35754985755
Accuracy on 3k train:  0.0733333333333


In [21]:
alexnet_10k_fullclass.fit(alex_net_feat_10k, labels_10k_full)
pickle.dump( alexnet_10k_fullclass, open( "alexnet_10k_fullclass.p", "wb" ) )

### sift 10k + full class

In [22]:
sift10k_full_feat_train, sift10k_full_feat_test, sift10k_full_y_train, sift10k_full_y_test = train_test_split(sift_feat_10k, labels_10k_full, test_size=0.33, random_state=42)
sift_10k_fullclass.fit(sift10k_full_feat_train, sift10k_full_y_train)
pred_sift_10k_full = sift_10k_fullclass.predict(sift10k_full_feat_test)
print np.sum(pred_sift_10k_full == sift10k_full_y_test) / float(len(sift10k_full_y_test))


0.11396011396


In [23]:
pred_sift_10k_full = sift_10k_fullclass.predict(sift_feat_train)
print 'Accuracy on 3k train: ', np.sum(pred_sift_10k_full == output) / float(len(output))

Accuracy on 3k train:  0.042


In [24]:
sift_10k_fullclass.fit(sift_feat_10k, labels_10k_full)
pickle.dump( alexnet_10k_fullclass, open( "sift_10k_fullclass.p", "wb" ) )

### rfc classifier

In [162]:
##rfc classifier on alexnet features only 3k
alexnet_rfc_feat_train, alexnet_rfc_feat_test, alexnet_rfc_y_train, alexnet_rfc_y_test = train_test_split(alexnet_feat_train_orig, output, test_size=0.33, random_state=42)
alexnet_3k_rfc.fit(alexnet_rfc_feat_train, alexnet_rfc_y_train)
pred_alexnet_3k_rfc = alexnet_3k_rfc.predict(alexnet_rfc_feat_test)
print np.sum(pred_alexnet_3k_rfc == alexnet_rfc_y_test) / float(len(alexnet_rfc_y_test))

0.323232323232


In [163]:
alexnet_rfc_feat_train, alexnet_rfc_feat_test, alexnet_rfc_y_train, alexnet_rfc_y_test = train_test_split(alexnet_feat_train, labels_10k_full, test_size=0.33, random_state=42)
alexnet_3k_rfc.fit(alexnet_rfc_feat_train, alexnet_rfc_y_train)
pred_alexnet_3k_rfc = alexnet_3k_rfc.predict(alexnet_rfc_feat_test)
print np.sum(pred_alexnet_3k_rfc == alexnet_rfc_y_test) / float(len(alexnet_rfc_y_test))

0.318862275449


In [ ]:
##rfc classifier on alexnet features only 3k
alexnet_rfc_feat_train, alexnet_rfc_feat_test, alexnet_rfc_y_train, alexnet_rfc_y_test = train_test_split(alexnet_feat_train_orig, output, test_size=0.33, random_state=42)
alexnet_3k_rfc.fit(alexnet_rfc_feat_train, alexnet_rfc_y_train)
pred_alexnet_3k_rfc = alexnet_3k_rfc.predict(alexnet_rfc_feat_test)
print np.sum(pred_alexnet_3k_rfc == alexnet_rfc_y_test) / float(len(alexnet_rfc_y_test))

In [131]:
##rfc classifier on attribute features only 3k
att_rfc_feat_train, att_rfc_feat_test, att_rfc_y_train, att_rfc_y_test = train_test_split(attributes_feat_train, output, test_size=0.33, random_state=42)
att_3k_rfc.fit(att_rfc_feat_train, att_rfc_y_train)
pred_att_3k_rfc = att_3k_rfc.predict(att_rfc_feat_test)
print np.sum(pred_att_3k_rfc == att_rfc_y_test) / float(len(att_rfc_y_test))

0.266666666667


In [139]:
##meta model for rfc trained on atta nd rfc trained on att
meta_model2 = RFC(n_estimators = 400)

#predict for all training data
f1 = alexnet_3k_rfc.predict_proba(alexnet_feat_train_orig)
f2 = att_3k_rfc.predict_proba(attributes_feat_train)


In [140]:
print f1.shape
print f2.shape

(3000, 200)
(3000, 200)


In [141]:
meta_train_feats2 = np.concatenate((f1, f2), axis=1)

In [144]:
meta_train_feats2.shape

(3000, 400)

In [142]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(meta_train_feats2, output, test_size=0.33, random_state=42)
meta_model2.fit(x_train2, y_train2)
np.sum(meta_model2.predict(x_test2) == y_test2) / float(len(y_test2))

0.16464646464646465

In [145]:
##meta model for naive bayes trained on atta nd rfc trained on att
from sklearn.naive_bayes import MultinomialNB as MNB
meta_model3 = MNB()
x_train3, x_test3, y_train3, y_test3 = train_test_split(meta_train_feats2, output, test_size=0.33, random_state=42)
meta_model3.fit(x_train3, y_train3)
np.sum(meta_model3.predict(x_test3) == y_test3) / float(len(y_test3))

0.19595959595959597

### Meta Model

In [25]:
# gensim_att_3k_fullclass = pickle.load( open( "gensim_att_3k_fullclass.p", "rb" ) )

In [65]:
meta_model = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1)

lb = preprocessing.LabelBinarizer()
lb.fit(output)

f1 = alexnet_3k_log.predict(alexnet_feat_train)
f1= lb.transform(f1)

f2 = sift_3k_log.predict_proba(sift_feat_train)

f3 = alexnet_10k_fullclass.predict(alexnet_feat_train)
f3 = lb.transform(f3)

f4 = sift_10k_fullclass.predict_proba(sift_feat_train)

f5 = att_3k_fullclass.predict_proba(attributes_feat_train)

f6 = gensim_att_3k_fullclass.predict_proba(gensim_attributes_train)

meta_train_feats = np.concatenate((f1, f2, f3, f4, f5, f6), axis=1)


In [157]:

f1_test = alexnet_3k_log.predict(alexnet_feat_test)
f1_test= lb.transform(f1_test)
f2_test = sift_3k_log.predict_proba(sift_feat_test)
f3_test = alexnet_10k_fullclass.predict(alexnet_feat_test)
f3_test = lb.transform(f3_test)
f4_test = sift_10k_fullclass.predict_proba(sift_feat_test)
f5_test = att_3k_fullclass.predict_proba(attributes_feat_test)
f6_test = gensim_att_3k_fullclass.predict_proba(gensim_attributes_test)

meta_test_feats = np.concatenate((f1_test, f2_test, f3_test, f4_test, f5_test, f6_test), axis=1)

In [158]:
meta_test_feats.shape

(1000, 1200)

In [159]:
x_train, x_test, y_train, y_test = train_test_split(meta_train_feats, output, test_size=0.33, random_state=42)
meta_model.fit(x_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=-1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [160]:
np.sum(meta_model.predict(x_test) == y_test)/float(len(y_test))

#33%

0.99696969696969695

In [ ]:
meta_model.fit(meta_train_feats, output)

In [71]:
pred1 = meta_model.predict(meta_test_feats)

In [76]:
img = []
with open("CS5785-final-data/test.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel', delimiter='\n')
    for r in reader:
        img.append(r[0])

In [80]:
img = np.array(img)

In [93]:
submission = pd.DataFrame({"ID": img})
submission.loc[:,'Category'] = pred1

In [95]:
submission.to_csv("submission1.csv", index=False)

In [86]:
submission

array([['0000.jpg', '0001.jpg', '0002.jpg', ..., '0997.jpg', '0998.jpg',
        '0999.jpg'],
       ['lawn', 'badlands', 'roof', ..., 'bowling alley', 'jewelry shop',
        'hayfield']], 
      dtype='|S22')